# Housing sales Price and Venues Data Analysis in Jakarta

## A. Introduction

### A.1 Description and Discussion Background

Jakarta has most density in Indonesia with 9.6 million people live. It has 661,5 SqKm with 14.464 people per square kilometer. The city is divided to 5 borough East, West, South,North and Center Jakarta. As an Indonesian, I use my capital city in my project. As a capital city and a business city, Jakarta has high price real estates. If people want to live in Jakarta, it should cost around Rp. 2.000.0000 - Rp. 5.000.000 average per month to rent an apartment or a boarding house, and around Rp. 500.000.000 - Rp. 1.000.000.000 for an apartment, of course it depends on the location. When we start to think to live in Jakarta, we may want to choose the best location for us with vary venues like cafe, hospital, etc and low cost absolutely. However it is difficult to obtain the information to determine which real estate we should buy. Will the house really worth it or not.

When we consider all the parameter, we can create a information chart where the real estate is placed and each district is clutered according to the venues.

### A.2 Data Description

This is the data I will use:

- https://ekonomi.bisnis.com/read/20171110/48/708065/berikut-pergerakana-harga-properti-di-jakarta Average house price in jakarta
- Foursquare API to show venues around the borough
- Jakarta geojson from https://gist.github.com/heffergm/d261f9cf3b334b8e523ee7fe0b3ef583?short_path=8e72f60

## B. Methodology

I used github to store my database. My main data include Borough, average pricing house in Jakarta based on ekonomi.bisnis.com, latitude and longitude. I used python for my programming language cause it has a tons of library that can help me to analyze. I used folium to visualize geographic detail of Jakarta.



1. Creating dataframe from news data

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import folium
import requests
from shapely.geometry import Point, Polygon, LineString
import json

In [2]:

datas = [['East Jakarta',7900,-6.225,106.900],['West Jakarta',13200,-6.167,106.763],['North Jakarta',17100,-6.155,106.892],['South Jakarta',17900,-6.261,106.810],['Center Jakarta',18700,-6.2088,106.845]]
df = pd.DataFrame(datas)
df.columns = ['Borugh','Avg Price','Latitude','Longitude']
df

,Borugh,Avg Price,Latitude,Longitude
0,East Jakarta,7900,-6.2250,106.900
1,West Jakarta,13200,-6.1670,106.763
2,North Jakarta,17100,-6.1550,106.892
3,South Jakarta,17900,-6.2610,106.810
4,Center Jakarta,18700,-6.2088,106.845


2. Creating Jakarta map separated by sub-district

In [3]:
jakarta_lat = -6.2088
jakarta_long = 106.8456
state_geo = 'jakarta_by_kelurahan_geo.geojson'

m = folium.Map(location=[jakarta_lat,jakarta_long], zoom_start=12)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    fill_color='BuPu',
    fill_opacity=0.3,
    line_opacity=0.8
).add_to(m)

folium.LayerControl().add_to(m)

m

3. Marking every center of sub-district

In [4]:


with open('jakarta_by_kelurahan_geo.geojson') as f:
    data = json.load(f)

# data['features']
for feature in data['features']:
#     print(feature['geometry']['coordinates'][0][0])
    p = Polygon(feature['geometry']['coordinates'][0][0])
    point = p.centroid
    
    folium.CircleMarker(
        [point.y, point.x],
        radius=5,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)
m

4 Creating function for generate venues arround sub-district

In [5]:
CLIENT_ID = 'VF1SBTXRNXKEDALWBRLRE2NT2K0Z3WVTUD3TGPV2DA5HOB3B' # your Foursquare ID
CLIENT_SECRET = 'WQI03FWI4MIEQVY0PRJYDAUOZJA1P0COTYGYSMFVQ2VXLAEI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
categoryId='4bf58dd8d48988d13b941735' #category id for food category
categories=[
    {'name':'Movie Theater','code':'4bf58dd8d48988d17f941735'},
    {'name':'College Academic Building','code':'4bf58dd8d48988d198941735'},
    {'name':'Asian Restaurant','code':'4bf58dd8d48988d142941735'},
    {'name':'Indonesian Restaurant','code':'4deefc054765f83613cdba6f'},
    {'name':'Bar','code':'4bf58dd8d48988d116941735'},
    {'name':'Athletics & Sports','code':'4f4528bc4b90abdf24c9de85'},
    {'name':'School','code':'4bf58dd8d48988d13b941735'},
    {'name':'Animal Shelter','code':'4e52d2d203646f7c19daa8ae'},
    {'name':'Spiritual Center','code':'4bf58dd8d48988d131941735'},
    {'name':'ATM','code':'52f2ab2ebcbc57f1066b8b56'},
    {'name':'Bank','code':'4bf58dd8d48988d10a951735'},
    {'name':'Market','code':'50be8ee891d4fa8dcc7199a7'},
    {'name':'Bus Station','code':'4bf58dd8d48988d1fe931735'},
    {'name':'Train Station','code':'4bf58dd8d48988d129951735'},
    {'name':'Hospital','code':'4bf58dd8d48988d196941735'},
    {'name':'Pharmacy','code':'4bf58dd8d48988d10f951735'},
    {'name':'Garden','code':'4bf58dd8d48988d15a941735'}
]

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def generate_venue(lat,lng):
    ll = '{},{}'.format(lat,lng)
    sortByPopularity = '1'
    search_query = ''
    radius = 500
    print(search_query + ' .... OK!')
    url = 'https://api.foursquare.com/v2/venues/search?categoryId={}&ll={}&sortByPopularity={}&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(categoryId,ll,sortByPopularity,CLIENT_ID, CLIENT_SECRET, jakarta_lat, jakarta_long, VERSION, search_query, radius, LIMIT)

    
    results = requests.get(url).json()
    
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    
    
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#     for lati, long, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
#         folium.CircleMarker(
#             [lati, long],
#             radius=5,
#             color='blue',
#             popup=label,
#             fill = True,
#             fill_color='blue',
#             fill_opacity=0.6
#         ).add_to(m)
    
    return dataframe

Your credentails:
CLIENT_ID: VF1SBTXRNXKEDALWBRLRE2NT2K0Z3WVTUD3TGPV2DA5HOB3B
CLIENT_SECRET:WQI03FWI4MIEQVY0PRJYDAUOZJA1P0COTYGYSMFVQ2VXLAEI


4.2 Generate function to store categorical venues in dataframe

In [8]:
def generate_url(lat,lng,categoryId):
    CLIENT_ID = 'VF1SBTXRNXKEDALWBRLRE2NT2K0Z3WVTUD3TGPV2DA5HOB3B' # your Foursquare ID
    CLIENT_SECRET = 'PKKKM3LVQ2LQXTX4G0TV13ULTMPFYNJO3NNXQHFPAPIAEAB1' # your Foursquare Secret
    VERSION = '20180604'
    LIMIT = 500
    ll = '{},{}'.format(lat,lng)
    sortByPopularity = '1'
    search_query = ''
    radius = 500
    url = 'https://api.foursquare.com/v2/venues/search?categoryId={}&ll={}&sortByPopularity={}&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(categoryId,ll,sortByPopularity,CLIENT_ID, CLIENT_SECRET, jakarta_lat, jakarta_long, VERSION, search_query, radius, LIMIT)
    return url

def find_venues(lat,lng,subDistrictName):
    environments = pd.DataFrame()
    print('Finding venues arround {}'.format(subDistrictName))
    for category in categories:
        url = generate_url(lat,lng,category['code'])
        results = requests.get(url).json()
#         print(results)
        
        try:
            venues = results['response']['venues']
            dataframe = json_normalize(venues)
            environment = dataframe[['name','location.lat','location.lng']]
            environment['sub-district'] = subDistrictName
            environment['sd-lat'] = lat
            environment['sd-lng'] = lng
            environment['category'] = category['name']
            environments = environments.append(environment)
            print('{} has {} {}'.format(subDistrictName,len(environments.index),category['name']))
        except :
            print("{} doesn't has {}".format(subDistrictName,category['name']))
            
            
            
        
    return environments
    

5. Store every sub-district venue to df

In [9]:
df_venues =  pd.DataFrame()
for feature in data['features']:
    df_temp = pd.DataFrame()
    p = Polygon(feature['geometry']['coordinates'][0][0])
    point = p.centroid
    df_temp = find_venues(point.y,point.x,feature['properties']['name'])
    df_venues = df_venues.append(df_temp)

df_venues.head()

Finding venues arround ANGKE


<ipython-input-8-cdd7baef5599>:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)
<ipython-input-8-cdd7baef5599>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  environment['sub-district'] = subDistrictName
<ipython-input-8-cdd7baef5599>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  environment['sd-lat'] = lat
<ipython-input-8-cdd7baef5599>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

ANGKE has 5 Movie Theater
ANGKE has 13 College Academic Building
ANGKE has 61 Asian Restaurant
ANGKE has 107 Indonesian Restaurant
ANGKE has 113 Bar
ANGKE has 129 Athletics & Sports
ANGKE has 163 School
ANGKE doesn't has Animal Shelter
ANGKE has 210 Spiritual Center
ANGKE doesn't has ATM
ANGKE has 227 Bank
ANGKE has 233 Market
ANGKE has 239 Bus Station
ANGKE has 242 Train Station
ANGKE has 247 Hospital
ANGKE has 255 Pharmacy
ANGKE doesn't has Garden
Finding venues arround PASEBAN
PASEBAN has 2 Movie Theater
PASEBAN has 16 College Academic Building
PASEBAN has 62 Asian Restaurant
PASEBAN has 107 Indonesian Restaurant
PASEBAN has 112 Bar
PASEBAN has 130 Athletics & Sports
PASEBAN has 140 School
PASEBAN doesn't has Animal Shelter
PASEBAN has 186 Spiritual Center
PASEBAN has 187 ATM
PASEBAN has 209 Bank
PASEBAN has 214 Market
PASEBAN has 224 Bus Station
PASEBAN has 237 Train Station
PASEBAN has 267 Hospital
PASEBAN has 272 Pharmacy
PASEBAN has 273 Garden
Finding venues arround KRENDANG
KRE

ANCOL has 187 ATM
ANCOL has 209 Bank
ANCOL has 214 Market
ANCOL has 224 Bus Station
ANCOL has 237 Train Station
ANCOL has 267 Hospital
ANCOL has 272 Pharmacy
ANCOL has 273 Garden
Finding venues arround BALE KAMBANG
BALE KAMBANG has 3 Movie Theater
BALE KAMBANG has 10 College Academic Building
BALE KAMBANG has 57 Asian Restaurant
BALE KAMBANG has 100 Indonesian Restaurant
BALE KAMBANG has 102 Bar
BALE KAMBANG has 109 Athletics & Sports
BALE KAMBANG has 125 School
BALE KAMBANG doesn't has Animal Shelter
BALE KAMBANG has 142 Spiritual Center
BALE KAMBANG has 144 ATM
BALE KAMBANG has 153 Bank
BALE KAMBANG doesn't has Market
BALE KAMBANG doesn't has Bus Station
BALE KAMBANG doesn't has Train Station
BALE KAMBANG has 160 Hospital
BALE KAMBANG has 165 Pharmacy
BALE KAMBANG has 167 Garden
Finding venues arround BALI MESTER
BALI MESTER has 2 Movie Theater
BALI MESTER has 16 College Academic Building
BALI MESTER has 62 Asian Restaurant
BALI MESTER has 107 Indonesian Restaurant
BALI MESTER has 11

CENGKARENG BARAT has 16 College Academic Building
CENGKARENG BARAT has 62 Asian Restaurant
CENGKARENG BARAT has 107 Indonesian Restaurant
CENGKARENG BARAT has 112 Bar
CENGKARENG BARAT has 130 Athletics & Sports
CENGKARENG BARAT has 140 School
CENGKARENG BARAT doesn't has Animal Shelter
CENGKARENG BARAT has 186 Spiritual Center
CENGKARENG BARAT has 187 ATM
CENGKARENG BARAT has 209 Bank
CENGKARENG BARAT has 214 Market
CENGKARENG BARAT has 224 Bus Station
CENGKARENG BARAT has 237 Train Station
CENGKARENG BARAT has 267 Hospital
CENGKARENG BARAT has 272 Pharmacy
CENGKARENG BARAT has 273 Garden
Finding venues arround CENGKARENG TIMUR
CENGKARENG TIMUR has 2 Movie Theater
CENGKARENG TIMUR has 16 College Academic Building
CENGKARENG TIMUR has 62 Asian Restaurant
CENGKARENG TIMUR has 107 Indonesian Restaurant
CENGKARENG TIMUR has 112 Bar
CENGKARENG TIMUR has 130 Athletics & Sports
CENGKARENG TIMUR has 140 School
CENGKARENG TIMUR doesn't has Animal Shelter
CENGKARENG TIMUR has 186 Spiritual Cente

...

RAGUNAN has 267 Hospital
RAGUNAN has 272 Pharmacy
RAGUNAN has 273 Garden
Finding venues arround SUSUKAN
SUSUKAN has 2 Movie Theater
SUSUKAN has 16 College Academic Building
SUSUKAN has 62 Asian Restaurant
SUSUKAN has 107 Indonesian Restaurant
SUSUKAN has 112 Bar
SUSUKAN has 130 Athletics & Sports
SUSUKAN has 140 School
SUSUKAN doesn't has Animal Shelter
SUSUKAN has 186 Spiritual Center
SUSUKAN has 187 ATM
SUSUKAN has 209 Bank
SUSUKAN has 214 Market
SUSUKAN has 224 Bus Station
SUSUKAN has 237 Train Station
SUSUKAN has 267 Hospital
SUSUKAN has 272 Pharmacy
SUSUKAN has 273 Garden
Finding venues arround RAMBUTAN
RAMBUTAN doesn't has Movie Theater
RAMBUTAN has 4 College Academic Building
RAMBUTAN has 19 Asian Restaurant
RAMBUTAN has 29 Indonesian Restaurant
RAMBUTAN has 31 Bar
RAMBUTAN has 32 Athletics & Sports
RAMBUTAN has 39 School
RAMBUTAN doesn't has Animal Shelter
RAMBUTAN has 53 Spiritual Center
RAMBUTAN doesn't has ATM
RAMBUTAN has 54 Bank
RAMBUTAN doesn't has Market
RAMBUTAN has 68 

SEMANAN has 62 Asian Restaurant
SEMANAN has 107 Indonesian Restaurant
SEMANAN has 112 Bar
SEMANAN has 130 Athletics & Sports
SEMANAN has 140 School
SEMANAN doesn't has Animal Shelter
SEMANAN has 186 Spiritual Center
SEMANAN has 187 ATM
SEMANAN has 209 Bank
SEMANAN has 214 Market
SEMANAN has 224 Bus Station
SEMANAN has 237 Train Station
SEMANAN has 267 Hospital
SEMANAN has 272 Pharmacy
SEMANAN has 273 Garden
Finding venues arround SEMPER BARAT
SEMPER BARAT has 2 Movie Theater
SEMPER BARAT has 16 College Academic Building
SEMPER BARAT has 62 Asian Restaurant
SEMPER BARAT has 107 Indonesian Restaurant
SEMPER BARAT has 112 Bar
SEMPER BARAT has 130 Athletics & Sports
SEMPER BARAT has 140 School
SEMPER BARAT doesn't has Animal Shelter
SEMPER BARAT has 186 Spiritual Center
SEMPER BARAT has 187 ATM
SEMPER BARAT has 209 Bank
SEMPER BARAT has 214 Market
SEMPER BARAT has 224 Bus Station
SEMPER BARAT has 237 Train Station
SEMPER BARAT has 267 Hospital
SEMPER BARAT has 272 Pharmacy
SEMPER BARAT has

SUNTER AGUNG has 140 School
SUNTER AGUNG doesn't has Animal Shelter
SUNTER AGUNG has 186 Spiritual Center
SUNTER AGUNG has 187 ATM
SUNTER AGUNG has 209 Bank
SUNTER AGUNG has 214 Market
SUNTER AGUNG has 224 Bus Station
SUNTER AGUNG has 237 Train Station
SUNTER AGUNG has 267 Hospital
SUNTER AGUNG has 272 Pharmacy
SUNTER AGUNG has 273 Garden
Finding venues arround SUNTER JAYA
SUNTER JAYA has 2 Movie Theater
SUNTER JAYA has 5 College Academic Building
SUNTER JAYA has 52 Asian Restaurant
SUNTER JAYA has 100 Indonesian Restaurant
SUNTER JAYA has 104 Bar
SUNTER JAYA has 119 Athletics & Sports
SUNTER JAYA has 129 School
SUNTER JAYA doesn't has Animal Shelter
SUNTER JAYA has 161 Spiritual Center
SUNTER JAYA doesn't has ATM
SUNTER JAYA has 179 Bank
SUNTER JAYA has 180 Market
SUNTER JAYA has 182 Bus Station
SUNTER JAYA doesn't has Train Station
SUNTER JAYA has 185 Hospital
SUNTER JAYA has 198 Pharmacy
SUNTER JAYA has 204 Garden
Finding venues arround TANGKI
TANGKI has 2 Movie Theater
TANGKI has 1

TUGU UTARA has 107 Indonesian Restaurant
TUGU UTARA has 112 Bar
TUGU UTARA has 130 Athletics & Sports
TUGU UTARA has 140 School
TUGU UTARA doesn't has Animal Shelter
TUGU UTARA has 186 Spiritual Center
TUGU UTARA has 187 ATM
TUGU UTARA has 209 Bank
TUGU UTARA has 214 Market
TUGU UTARA has 224 Bus Station
TUGU UTARA has 237 Train Station
TUGU UTARA has 267 Hospital
TUGU UTARA has 272 Pharmacy
TUGU UTARA has 273 Garden
Finding venues arround UJUNG MENTENG
UJUNG MENTENG has 2 Movie Theater
UJUNG MENTENG has 16 College Academic Building
UJUNG MENTENG has 62 Asian Restaurant
UJUNG MENTENG has 107 Indonesian Restaurant
UJUNG MENTENG has 112 Bar
UJUNG MENTENG has 130 Athletics & Sports
UJUNG MENTENG has 140 School
UJUNG MENTENG doesn't has Animal Shelter
UJUNG MENTENG has 186 Spiritual Center
UJUNG MENTENG has 187 ATM
UJUNG MENTENG has 209 Bank
UJUNG MENTENG has 214 Market
UJUNG MENTENG has 224 Bus Station
UJUNG MENTENG has 237 Train Station
UJUNG MENTENG has 267 Hospital
UJUNG MENTENG has 27

,name,location.lat,location.lng,sub-district,sd-lat,sd-lng,category
0,Bengkel Ko Aji,-6.145625,106.790205,ANGKE,-6.145663,106.795697,Movie Theater
1,TV Room,-6.145872,106.796391,ANGKE,-6.145663,106.795697,Movie Theater
2,Season City XX1,-6.149000,106.794503,ANGKE,-6.145663,106.795697,Movie Theater
3,My Cinema,-6.145094,106.798164,ANGKE,-6.145663,106.795697,Movie Theater
4,XXI Junction,-6.140917,106.794491,ANGKE,-6.145663,106.795697,Movie Theater


In [14]:
df_venues.describe()

,location.lat,location.lng,sd-lat,sd-lng
count,67030.000000,67030.000000,67030.000000,67030.000000
mean,-6.208807,106.841042,-6.206611,106.835393
std,0.026465,0.025605,0.057940,0.051766
min,-6.354669,106.710945,-6.357019,106.698753
25%,-6.212426,106.842783,-6.243530,106.803426
50%,-6.209293,106.845054,-6.200907,106.835155
75%,-6.206641,106.847536,-6.162798,106.868810
max,-6.108841,106.956639,-6.102968,106.963465


In [18]:
df_venues.drop(columns='index',inplace=True)

In [19]:
df_venues.to_csv('venues_2.csv')

In [ ]:
data['features'][0]['properties']['name']

5. Marking every venues in current district

In [ ]:
for feature in data['features']:
#     print(feature['geometry']['coordinates'][0][0])
    p = Polygon(feature['geometry']['coordinates'][0][0])
    point = p.centroid
    generate_venue(point.y,point.x)

In [ ]:
venues_around = generate_venue(-6.2250,106.900)
venues_around

In [ ]:
venues_around['categories'][0]

In [ ]:
CLIENT_ID = 'VF1SBTXRNXKEDALWBRLRE2NT2K0Z3WVTUD3TGPV2DA5HOB3B' # your Foursquare ID
CLIENT_SECRET = 'WQI03FWI4MIEQVY0PRJYDAUOZJA1P0COTYGYSMFVQ2VXLAEI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
categoryId='4d4b7105d754a06374d81259' #category id for food category
near='Jembatan + lima'
ll = '-6.2250,106.900'
sortByPopularity = '1'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

search_query = ''
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?categoryId={}&ll={}&sortByPopularity={}&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(categoryId,ll,sortByPopularity,CLIENT_ID, CLIENT_SECRET, jakarta_lat, jakarta_long, VERSION, search_query, radius, LIMIT)
url

In [ ]:
venue_id = '4fd81ed3e4b0859a9e5f25bc' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

results = requests.get(url).json()

try:
    print(results)
except:
    print('This venue has not been rated yet.')
    
results['response']['venue']['tips']['count']


In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [ ]:
dataframe_filtered.name

In [ ]:
venues_map = folium.Map(location=[jakarta_lat, jakarta_long], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [jakarta_lat, jakarta_long],
    radius=10,
    color='red',
    popup='',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(m)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

# display map
m

In [ ]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')